In [1]:
from transformers import AutoModelForMultipleChoice, AutoTokenizer
import torch
from utils import util

# Load models
checkpoint_path = "/Users/alvinchen/Documents/GitHub/brainteaser-data/testing/models/bert-BASED-uncased/"
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
model = AutoModelForMultipleChoice.from_pretrained(checkpoint_path)

In [2]:
# Load datasets
dataset, keys = util.load_data()
test = dataset[1]
valid = dataset[2][-101:]

Loaded WP-train.npy at index 0
Loaded SP_eval_data_for_practice.npy at index 1
Loaded SP-train.npy at index 2
Loaded WP_eval_data_for_practice.npy at index 3


In [3]:
# Evaluate dev set
def predict(model, tokenizer, dataset):
    preds = []
    model.eval()
    for sample in dataset:
        inputs = [[sample["question"], answer] for answer in sample["choice_list"]]
        tokens = tokenizer(inputs, return_tensors="pt", padding=True)
        labels = torch.tensor(0).unsqueeze(0)
        outputs = model(**{k: v.unsqueeze(0) for k, v in tokens.items()}, labels=labels)
        logits = outputs.logits
        pred = logits.argmax().item()
        preds.append(pred)
    return preds

preds = predict(model, tokenizer, valid)
valid_labs = [int(i["label"]) for i in valid]

In [4]:
f1 = util.avg_f1_score(preds, valid_labs, [0,1,2,3])
accuracy = util.accuracy(preds, valid_labs)
print("Accuracy: ", accuracy)
print("F1: ", f1)

Accuracy:  0.6732673267326733
F1:  0.6693775243525777


120
